In [28]:
from dataclasses import dataclass
from datetime import datetime
from typing import Generic, TypeVar

import pandas as pd

T = TypeVar("T")

@dataclass
class Observation(Generic[T]):
    timestamp: datetime
    value: T
        
    def to_df(self):
        return pd.DataFrame(self.to_dict())
        
    def to_dict(self):
        return {
            "timestamp": self.timestamp,
            "value": self.value.to_dict(),
        }

In [29]:
from dataclasses import dataclass

import pandas as pd

@dataclass
class Candle(object):
    open: float
    high: float
    low: float
    close: float
    volume: float
        
    def to_df(self):
        return pd.DataFrame(self.__to_dict())
    
    def to_dict(self):
        return {
            "open": self.open,
            "high": self.high,
            "low": self.low,
            "close": self.close,
            "volume": self.volume,
        }

In [30]:
def typical_price(candle: Candle) -> float:
    return (candle.high + candle.low + candle.close) / 3

In [31]:
def cumvol(candles: list[Observation[Candle]]) -> float:
    return sum([x.value.volume for x in candles])

In [32]:
def tradingview_vwap(obs: list[Observation[Candle]]) -> list[Observation[float]]:
    return [
        Observation(
            x.timestamp,
            sum([typical_price(x.value) * x.value.volume for x in obs]) / cumvol(obs)
        )
        for x in obs
    ]

In [33]:
import requests

def get_candles_dydx(ticker: str) -> list[Observation[Candle]]:
    return [
        Observation(
            # this is really gross, hacky, and just generally upsetting -- dYdX uses a non-ISO8601 compliant timestamp so we have to truncate the string ourselves
            datetime.fromisoformat(x["startedAt"][:x["startedAt"].find('.')]),
            Candle(
                float(x["open"]),
                float(x["high"]),
                float(x["low"]),
                float(x["close"]),
                float(x["usdVolume"])
            )
        )
        for x in requests.get(f"https://api.dydx.exchange/v3/candles/{ticker}?resolution=1MIN").json()["candles"]]

In [34]:
dydx_eth_prices = get_candles_dydx("ETH-USD")

In [35]:
def obs2df(xs: list[Observation[T]]) -> pd.DataFrame:
    return pd.DataFrame({ 
        "timestamp": [x.timestamp for x in xs],
        "open": [x.value.open for x in xs],
        "high": [x.value.high for x in xs],
        "low": [x.value.low for x in xs],
        "close": [x.value.close for x in xs],
        "volume": [x.value.volume for x in xs],
    })

In [36]:
def vwap_df(vwap: list[Observation[float]]) -> pd.DataFrame:
    return pd.DataFrame({
        "timestamp": [x.timestamp for x in vwap],
        "vwap": [x.value for x in vwap],
    })

In [40]:
import finplot as fplt
import pandas as pd

def plot_candles(candles: list[Observation[Candle]]):
    df = obs2df(candles)
    fplt.candlestick_ochl(df[["timestamp", "open", "close", "high", "low"]])
    fplt.show()

In [41]:
import finplot as fplt
import pandas as pd

def plot_candles_with_vwap(candles: list[Observation[Candle]]):
    df = obs2df(candles)
    fplt.candlestick_ochl(df[["timestamp", "open", "close", "high", "low"]])
    fplt.plot(vwap_df(tradingview_vwap(candles)))
    fplt.show()

In [42]:
plot_candles_with_vwap(dydx_eth_prices)